In [1]:
# importing necessary libraries 
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np
from string import punctuation
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# importing neural network libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, LSTM, RNN, SpatialDropout1D

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_data = train.copy()
test_data = test.copy()

In [ ]:
print(tf.__version__)

1.14.0


In [ ]:
train_data = train_data.set_index('id', drop = True)
print(train_data.shape)
train_data.head()

(20800, 4)


,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
print(test_data.shape)
test_data.head()

(5200, 4)


,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [ ]:
# checking for missing values
train_data.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# dropping missing values from text columns alone. 

train_data[['title', 'author']] = train_data[['title', 'author']].fillna(value = 'Missing')
print(train_data)


                                                   title  \
id                                                         
0      House Dem Aide: We Didn’t Even See Comey’s Let...   
1      FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2                      Why the Truth Might Get You Fired   
3      15 Civilians Killed In Single US Airstrike Hav...   
4      Iranian woman jailed for fictional unpublished...   
...                                                  ...   
20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799                          What Keeps the F-35 Alive   

                                          author  \
id                                                 
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2          

In [ ]:
train_data = train_data.dropna()
train_data.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
length = []
[length.append(len(str(text))) for text in train_data['text']]
train_data['length'] = length
train_data.head()

,title,author,text,label,length
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,4160
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,938


In [ ]:
min(train_data['length']), max(train_data['length']), round(sum(train_data['length'])/len(train_data['length']))

(1, 142961, 4553)

In [ ]:
len(train_data[train_data['length'] < 50])
train_data['text'][train_data['length'] < 50]

id
82                                                   
169                                                  
173                                   Guest   Guest  
196            They got the heater turned up on high.
295                                                  
                             ...                     
20350                         I hope nobody got hurt!
20418                                 Guest   Guest  
20431    \nOctober 28, 2016 The Mothers by stclair by
20513                                                
20636                              Trump all the way!
Name: text, Length: 207, dtype: object

In [ ]:
# dropping the outliers
train_data = train_data.drop(train_data['text'][train_data['length'] < 50].index, axis = 0)
min(train_data['length']), max(train_data['length']), round(sum(train_data['length'])/len(train_data['length']))
train_data.head()

,title,author,text,label,length
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,4160
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,938


In [ ]:
# Tokenizing the text - converting the words, letters into counts or numbers. 
# We dont need to explicitly remove the punctuations. we have an inbuilt option in Tokenizer for this purpose

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras.utils import to_categorical
import pickle


max_features = 4500
MAX_SEQUENCE_LENGTH=1500 #max word length of each individual article
EMBEDDING_DIM=300 #dimensionality of the embedding vector (50, 100, 200, 300)

tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = train_data['text'])
sequences = tokenizer.texts_to_sequences(texts = train_data['text'])

# now applying padding to make them even shaped.
X = pad_sequences(sequences = sequences, maxlen = MAX_SEQUENCE_LENGTH, padding = 'pre')
print(X.shape)

labels = train_data['label'].values
Y = to_categorical(labels, num_classes=len(set(labels)))
print(Y.shape)

#word index
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Using TensorFlow backend.


(20554, 1500)
(20554, 2)
Found 238036 unique tokens.


In [ ]:
#split the data (90% train, 5% test, 5% validation)

train_text = X[:int(len(X)*0.9)]
print("training data text: ")
print(train_text)

train_labels = Y[:int(len(X)*0.9)]
print("training data labels: ")
print(train_labels)


training data text: 
[[   0    0    0 ... 1096    5 4240]
 [   0    0    0 ... 3757  138 1169]
 [   0    0    0 ...    3  208  670]
 ...
 [   0    0    0 ... 1665  327   11]
 [   0    0    0 ...    5  559   11]
 [   0    0    0 ...  535   27 1330]]
training data labels: 
[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [ ]:
test_text = X[int(len(X)*0.9):int(len(X)*0.95)]
print("testing data text: ")
print(test_text)

test_labels = Y[int(len(X)*0.9):int(len(X)*0.95)]
print("testing data labels: ")
print(test_labels)


testing data text: 
[[   0    0    0 ...  680 1770 1280]
 [   0    0    0 ... 4394   10  198]
 [   0    0    0 ...   47   27 3298]
 ...
 [   0    0    0 ...   41  130   14]
 [   0    0    0 ...  276 1052  178]
 [   0    0    0 ...   86   14  348]]
testing data labels: 
[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [ ]:
valid_text = X[int(len(X)*0.95):]
print("validation data text: ")
print(valid_text)

valid_labels = Y[int(len(X)*0.95):]
print("validation data labels: ")
print(valid_labels)

validation data text: 
[[   0    0    0 ...  176   11  534]
 [ 205  399   99 ...  945  691 4240]
 [   0    0    0 ...   16   13  111]
 ...
 [   0    0    0 ... 3774  774   11]
 [   0    0    0 ...   10   59 3818]
 [   0    0    0 ... 1543 2259 3539]]
validation data labels: 
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [ ]:
def load_embeddings(word_index, embeddingsfile='glove.6B.%id.txt' %EMBEDDING_DIM):
    embeddings_index = {}
    f = open(embeddingsfile, 'r', encoding='utf8')
    for line in f:
        #here we parse the data from the file
        values = line.split(' ') #split the line by spaces
        word = values[0] #each line starts with the word
        coefs = np.asarray(values[1:], dtype='float32') #the rest of the line is the vector
        embeddings_index[word] = coefs #put into embedding dictionary
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    
    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)
    return embedding_layer
    
#and build the embedding layer
embedding_layer = load_embeddings(word_index)

#build the baseline model architecture
from keras import Sequential, Model, Input
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Flatten, Dense, GlobalAveragePooling1D, Dropout, LSTM, CuDNNLSTM, RNN, SimpleRNN, Conv2D, GlobalMaxPooling1D
from keras import callbacks

def baseline_model(sequence_input, embedded_sequences, classes=2):
    x = Conv1D(64, 5, activation='relu')(embedded_sequences)
    x = MaxPooling1D(5)(x)
    x = Conv1D(128, 3, activation='relu')(x)
    x = MaxPooling1D(5)(x)
    x = Conv1D(256, 2, activation='relu')(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(classes, activation='softmax')(x)

    model = Model(sequence_input, preds)
    return model


#put embedding layer into input of the model
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

model = baseline_model(sequence_input, embedded_sequences, classes=2)

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['acc'])

print(model.summary())

model.fit(train_text, train_labels,
          validation_data=(valid_text, valid_labels),
          epochs=5, batch_size=64)

Found 400000 word vectors.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1500, 300)         71411100  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1496, 64)          96064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 299, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 297, 128)          24704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (N

In [ ]:
#test the model with the testing data (5%)
model.evaluate(test_text,test_labels)


1028/1028 [==============================] - 5s 5ms/step


[0.15206673538044152, 0.9533073902130127]

In [ ]:
from urllib.request import urlopen 
from newspaper import fulltext
import requests 

def tokenize_text(text1):
    sequences = tokenizer.texts_to_sequences(text1)
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    return data
    
# top US news api 
main_url = "http://newsapi.org/v2/top-headlines?country=us&apiKey=aeb02e73e782448cb7f027138a35f496"

# fetching data in json format 
tech = requests.get(main_url).json()
    
# getting all articles in a string article 
article = tech["articles"]
  
# empty list which will  
# contain all trending news 
results = [] 
      
for ar in article: 
    results.append(ar["url"]) 
        
print ("List value are : ") 


i=0
label_list = []

prediction = []
for i in range(len(results)):
    try:
        print(results[i])
        html = requests.get(results[i]).text
        text1 = fulltext(html)
        tok = tokenize_text([text1])
        print(tok)
        preds = model.predict(tok)
        pred1 = preds[0]
        print(pred1)
        true = pred1[0]
        p = 0
        if true < 0.50:
            p = 0
        else:
            p = 1
        label_list.append(p)
        print(p)
    except Exception as e:
        print('error' +str(e))
        p=0
        label_list.append(p)

List value are : 
https://www.cnn.com/2020/07/01/china/hong-kong-national-security-law-july-1-intl-hnk/index.html
[[   0    0    0 ...   43    3 1891]]
[0.9515556  0.04844436]
1
https://www.cnbc.com/2020/06/30/stock-market-futures-open-to-close-news.html
[[   0    0    0 ...    6 2185    1]]
[0.1821113  0.81788874]
0
https://www.cnet.com/how-to/spacex-just-launched-a-space-force-satellite-with-brand-new-falcon-9-booster/
[[   0    0    0 ...  548    2 1238]]
[0.99826175 0.00173824]
1
https://www.youtube.com/watch?v=G2D7DAzp2_g
error'NoneType' object has no attribute 'xpath'
https://www.cnn.com/2020/07/01/politics/donald-trump-coronavirus/index.html
[[   0    0    0 ... 3024  249   17]]
[0.9983334  0.00166668]
1
https://www.cnn.com/2020/06/30/politics/primary-results-highlights-june-30/index.html
[[   0    0    0 ...   12 1437  467]]
[0.99843997 0.00156009]
1
https://www.geo.tv/latest/295694-kate-middleton-warned-prince-harry-to-move-with-caution-while-dating-meghan-markle
[[   0    0  

In [ ]:
print(results)
print(len(results))
print(label_list)
print(len(label_list))

['https://www.cnn.com/2020/07/01/china/hong-kong-national-security-law-july-1-intl-hnk/index.html', 'https://www.cnbc.com/2020/06/30/stock-market-futures-open-to-close-news.html', 'https://www.cnet.com/how-to/spacex-just-launched-a-space-force-satellite-with-brand-new-falcon-9-booster/', 'https://www.youtube.com/watch?v=G2D7DAzp2_g', 'https://www.cnn.com/2020/07/01/politics/donald-trump-coronavirus/index.html', 'https://www.cnn.com/2020/06/30/politics/primary-results-highlights-june-30/index.html', 'https://www.geo.tv/latest/295694-kate-middleton-warned-prince-harry-to-move-with-caution-while-dating-meghan-markle', 'https://www.marketwatch.com/story/a-new-virus-emerging-in-chinese-pig-farms-has-the-essential-hallmarks-for-a-pandemic-2020-07-01', 'https://www.engadget.com/mars-2020-rover-launch-delay-july-30-002955836.html', 'https://phys.org/news/2020-06-zealand-ancient-monster-penguins-northern.html', 'https://9to5mac.com/2020/06/30/new-evilquest-mac-ransomware-found-in-pirated-apps-e

In [ ]:
# making an updated list of all url
R = len(results)
C = 1
  
# Initialize matrix 
url = [] 
print("Enter the entries rowwise:") 

for i in range(R):
    url.append(results[i])
    
# For printing the matrix 
for i in range(R):
    print(url[i], end = " ") 
    print()

Enter the entries rowwise:
https://www.cnn.com/2020/07/01/china/hong-kong-national-security-law-july-1-intl-hnk/index.html 
https://www.cnbc.com/2020/06/30/stock-market-futures-open-to-close-news.html 
https://www.cnet.com/how-to/spacex-just-launched-a-space-force-satellite-with-brand-new-falcon-9-booster/ 
https://www.youtube.com/watch?v=G2D7DAzp2_g 
https://www.cnn.com/2020/07/01/politics/donald-trump-coronavirus/index.html 
https://www.cnn.com/2020/06/30/politics/primary-results-highlights-june-30/index.html 
https://www.geo.tv/latest/295694-kate-middleton-warned-prince-harry-to-move-with-caution-while-dating-meghan-markle 
https://www.marketwatch.com/story/a-new-virus-emerging-in-chinese-pig-farms-has-the-essential-hallmarks-for-a-pandemic-2020-07-01 
https://www.engadget.com/mars-2020-rover-launch-delay-july-30-002955836.html 
https://phys.org/news/2020-06-zealand-ancient-monster-penguins-northern.html 
https://9to5mac.com/2020/06/30/new-evilquest-mac-ransomware-found-in-pirated-a

In [ ]:
# making a list of all prediction values
R = len(label_list)
C = 1
  
# Initialize matrix 
label = [] 
print("Enter the entries rowwise:") 
  
# For user input 
for i in range(R):
    label.append(label_list[i]) 
# For printing the matrix 
for i in range(R): 
    print(label[i], end = " ") 
    print() 

Enter the entries rowwise:
1 
0 
1 
0 
1 
1 
1 
1 
0 
0 
1 
1 
1 
0 
1 
1 
1 
0 
0 
1 


In [ ]:
d = {'URL':url,'LABEL':label}
print(d)

{'URL': ['https://www.cnn.com/2020/07/01/china/hong-kong-national-security-law-july-1-intl-hnk/index.html', 'https://www.cnbc.com/2020/06/30/stock-market-futures-open-to-close-news.html', 'https://www.cnet.com/how-to/spacex-just-launched-a-space-force-satellite-with-brand-new-falcon-9-booster/', 'https://www.youtube.com/watch?v=G2D7DAzp2_g', 'https://www.cnn.com/2020/07/01/politics/donald-trump-coronavirus/index.html', 'https://www.cnn.com/2020/06/30/politics/primary-results-highlights-june-30/index.html', 'https://www.geo.tv/latest/295694-kate-middleton-warned-prince-harry-to-move-with-caution-while-dating-meghan-markle', 'https://www.marketwatch.com/story/a-new-virus-emerging-in-chinese-pig-farms-has-the-essential-hallmarks-for-a-pandemic-2020-07-01', 'https://www.engadget.com/mars-2020-rover-launch-delay-july-30-002955836.html', 'https://phys.org/news/2020-06-zealand-ancient-monster-penguins-northern.html', 'https://9to5mac.com/2020/06/30/new-evilquest-mac-ransomware-found-in-pirate

In [ ]:
df=pd.DataFrame(d)
print(df)

                                                  URL  LABEL
0   https://www.cnn.com/2020/07/01/china/hong-kong...      1
1   https://www.cnbc.com/2020/06/30/stock-market-f...      0
2   https://www.cnet.com/how-to/spacex-just-launch...      1
3         https://www.youtube.com/watch?v=G2D7DAzp2_g      0
4   https://www.cnn.com/2020/07/01/politics/donald...      1
5   https://www.cnn.com/2020/06/30/politics/primar...      1
6   https://www.geo.tv/latest/295694-kate-middleto...      1
7   https://www.marketwatch.com/story/a-new-virus-...      1
8   https://www.engadget.com/mars-2020-rover-launc...      0
9   https://phys.org/news/2020-06-zealand-ancient-...      0
10  https://9to5mac.com/2020/06/30/new-evilquest-m...      1
11  https://www.dailymail.co.uk/tvshowbiz/article-...      1
12  https://www.foxnews.com/sports/baseballs-minor...      1
13  https://www.theguardian.com/us-news/2020/jun/3...      0
14  https://www.nytimes.com/2020/06/30/us/coronavi...      1
15  https://www.foxnews.

In [ ]:
df.dtypes

URL      object
LABEL     int64
dtype: object

In [ ]:
df = df[df['LABEL'] == True]
print(df)

                                                  URL  LABEL
0   https://www.cnn.com/2020/07/01/china/hong-kong...      1
2   https://www.cnet.com/how-to/spacex-just-launch...      1
4   https://www.cnn.com/2020/07/01/politics/donald...      1
5   https://www.cnn.com/2020/06/30/politics/primar...      1
6   https://www.geo.tv/latest/295694-kate-middleto...      1
7   https://www.marketwatch.com/story/a-new-virus-...      1
10  https://9to5mac.com/2020/06/30/new-evilquest-m...      1
11  https://www.dailymail.co.uk/tvshowbiz/article-...      1
12  https://www.foxnews.com/sports/baseballs-minor...      1
14  https://www.nytimes.com/2020/06/30/us/coronavi...      1
15  https://www.foxnews.com/media/adam-kinzinger-r...      1
16  https://www.foxnews.com/politics/pompeo-united...      1
19  https://www.washingtonpost.com/health/protests...      1


In [ ]:
print(df['URL'])

0     https://www.cnn.com/2020/07/01/china/hong-kong...
2     https://www.cnet.com/how-to/spacex-just-launch...
4     https://www.cnn.com/2020/07/01/politics/donald...
5     https://www.cnn.com/2020/06/30/politics/primar...
6     https://www.geo.tv/latest/295694-kate-middleto...
7     https://www.marketwatch.com/story/a-new-virus-...
10    https://9to5mac.com/2020/06/30/new-evilquest-m...
11    https://www.dailymail.co.uk/tvshowbiz/article-...
12    https://www.foxnews.com/sports/baseballs-minor...
14    https://www.nytimes.com/2020/06/30/us/coronavi...
15    https://www.foxnews.com/media/adam-kinzinger-r...
16    https://www.foxnews.com/politics/pompeo-united...
19    https://www.washingtonpost.com/health/protests...
Name: URL, dtype: object


In [ ]:
df1 = df[['URL']].to_numpy()
print(df1)

#df.to_csv ('/content/drive/My Drive/prediction.csv',mode='a', index = False, header=False)

[['https://www.reuters.com/article/us-health-coronavirus-china-idUSKCN21V06R']
 ['https://www.washingtonpost.com/world/2020/04/13/coronavirus-latest-news/']
 ['https://www.forbes.com/sites/startswithabang/2020/04/13/the-most-important-image-ever-taken-by-nasas-hubble-space-telescope/']
 ['https://indiagoneviral.com/entertainment/2020/04/13/killing-eve-season-premiere-recap-end-of-story/22471/randallbedford/']
 ['https://www.thenews.com.pk/latest/643777-ryan-reynolds-reignites-funny-feud-with-hugh-jackman']
 ['https://nypost.com/2020/04/12/smithfield-closes-sd-pork-plant-indefinitely-warns-of-meat-shortages/']
 ['http://www.marketwatch.com/story/alarming-surge-in-coronavirus-deaths-at-nursing-homes-nationwide-2020-04-12']
 ['https://www.usatoday.com/story/sports/nba/jazz/2020/04/12/rudy-gobert-donovan-mitchell-strained-relationship/2980754001/']
 ['https://www.nytimes.com/2020/04/12/arts/television/westworld-recap.html']
 ['https://thehill.com/homenews/administration/492449-trump-lashes

In [ ]:
type(df1)

numpy.ndarray

In [ ]:
import strconv as strconv
from flask import Flask, request, jsonify, render_template, send_file, \
    make_response  # import main Flask class and request object
#import cv2 
import json

app = Flask(__name__)

# root
@app.route("/")
def index():
    """
    this is a root dir of my server
    :return: str
    """
    return "This is root!!!!"

@app.route('/image', methods=["GET", "POST"])

def query_example():
    url_array = df1
    print(url_array)
    url_array = str(url_array)
    return jsonify({'result': url_array})


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8888, threaded=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8888/ (Press CTRL+C to quit)
